## CSCI 5654 - Linear Programming - Project

***
#### Team Members
#### 1. Ketan Ramesh
#### 2. Shreyas Gopalakrishna
***

## Vehicle Routing Problem with Simultaneous Pickup and Delivery

In [ ]:
!pip3 install pulp

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import pulp
import gurobipy as gp
from gurobipy import GRB
from scipy.spatial import distance

## Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP)
​
Paper Link - [Vehicle Routing Problem with Deliveries and Pickups: Modelling Issues and Meta-heuristics Solution Approaches](https://core.ac.uk/download/pdf/19477982.pdf)
​
<div align="justify">
The code provided below creates a <code>class</code> to frame and solve the Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) using the two-index flow formulation as mentioned in the paper. The formulation of the problem as a Mixed ILP is as follows:
</div>
​
$$
min \sum_{i=0}^{n} \sum_{j=0}^{n} d_{ij} x_{ij}
$$
​
$$
s.t. \sum_{i=0}^{n} x_{ij} = 1, j \in \{1, ..., n\}, - (3.1)
$$
​
$$
\sum_{i=0}^{n} x_{ji} = 1, j \in \{1, ..., n\}, - (3.2)
$$
​
$$
\sum_{i=0}^{n} R_{ij} - q_{j} = \sum_{i=0}^{n} R_{ji}, j \in \{1, ..., n\}, - (3.3)
$$
​
$$
\sum_{i=0}^{n} P_{ij} + b_{j} = \sum_{i=0}^{n} P_{ji}, j \in \{1, ..., n\}, - (3.4)
$$
​
$$
\sum_{i=1}^{n} R_{i0} = 0, - (3.5)
$$
​
$$
\sum_{i=1}^{n} P_{0i} = 0, - (3.6)
$$
​
$$
R_{ij} + P_{ij} \leq C x_{ij}, i, j \in \{0, ..., n\}, - (3.7)
$$
​
$$
x_{ij} = \{0, 1\}, i, j \in \{1, ... n\}
$$
​
$$
R_{ij}, P_{ij} \geq 0, i, j \in \{1, ..., n\}
$$
​
where,<br />
​
d<sub>ij</sub> = Cost to travel from node i to j.<br />
q<sub>i</sub> = The delivery demand of node i.<br />
b<sub>i</sub> = The pickup demand of node i.<br />
C = Vehicle capacity.<br />
​
<b>Decision Variables:</b><br />
1. x<sub>ij</sub> = {<b>1</b>, if edge (i, j) present in any route. <b>0</b>, otherwise}.<br />
2. R<sub>ij</sub> = The amount of delivery goods on board on arc ij.<br />
3. P<sub>ij</sub> = The amount of pickup goods on board on arc ij.<br />
​
<div align="justify">
The objective is to minimize the sum of the distances of all routes that satisfy the given constraints. The constraints (3.1) and (3.2) ensure that every node is visited exactly once (every node leads to only one other node, every node must be visited by only one other node). The constraints (3.3) and (3.4) ensure that the flow conservation is met (at a node <i>j</i>, the amount of delivery load after servicing delivery of node <i>j</i>, must be equal to pickup load at the same node. The pickup load condition is similar in nature). The constraints (3.5) and (3.6) ensure that the depot has zero pickup and delivery load. The constraint (3.7) ensures that the pickup and demand loads do not exceed the capacity of the vehicle at all nodes in the routes.  
</div>
​

In [ ]:
class VRPSDP_GUROBI:
	def __init__(self, costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle):
		self.costMatrix = costMatrix
		self.n = len(costMatrix)
		self.demand = demand
		self.pickup = pickup
		self.numberOfVehicles = numberOfVehicles
		self.capacityOfVehicle = capacityOfVehicle
		self.initialzeLP()

	def initialzeLP(self):
		self.cvrpLP = gp.Model('VRP-SDP')
		x, R, P = [], [], []

		# Create decision variables
		for i in range(self.n):
			xRow, RRow, PRow = [], [], []
			for j in range(self.n):
				xRow.append(self.cvrpLP.addVar(name='x('+str(i)+","+str(j)+")", vtype=GRB.BINARY, lb=0, ub=1))
				RRow.append(self.cvrpLP.addVar(name='R('+str(i)+","+str(j)+")", vtype=GRB.INTEGER, lb=0))
				PRow.append(self.cvrpLP.addVar(name='P('+str(i)+","+str(j)+")", vtype=GRB.INTEGER, lb=0))
			x.append(xRow)
			R.append(RRow)
			P.append(PRow)

		# Create objective
		objective = 0
		for i in range(self.n):
			for j in range(self.n):
				objective += self.costMatrix[i][j] * x[i][j]
		self.cvrpLP.setObjective(objective,GRB.MINIMIZE)

		# constraint 1
		for j in range(1, self.n):
			const1 = None
			for i in range(self.n):
				if(const1 == None):
					const1 = x[i][j]
				else:
					const1 = const1 + x[i][j]
			self.cvrpLP.addConstr(const1 == 1)
		
		# constraint 2
		for j in range(1, self.n):
			const2 = None
			for i in range(self.n):
				if(const2 == None):
					const2 = x[j][i]
				else:
					const2 = const2 + x[j][i]
			self.cvrpLP.addConstr(const2 == 1)

		# constraint 3
		for j in range(1, self.n):
			const3a, const3b = None, None
			for i in range(self.n):
				if(const3a == None):
					const3a = R[i][j]
				else:
					const3a = const3a + R[i][j]
				if(const3b == None):
					const3b = R[j][i]
				else:
					const3b = const3b + R[j][i]
			self.cvrpLP.addConstr(const3a - self.demand[j] == const3b)

		# constraint 4
		for j in range(1, self.n):
			const4a, const4b = None, None
			for i in range(self.n):
				if(const4a == None):
					const4a = P[i][j]
				else:
					const4a = const4a + P[i][j]
				if(const4b == None):
					const4b = P[j][i]
				else:
					const4b = const4b + P[j][i]
			self.cvrpLP.addConstr(const4a + self.pickup[j] == const4b)

		# constraint 5
		const5 = None
		for i in range(1, self.n):
			if(const5 == None):
				const5 = P[0][i]
			else:
				const5 = const5 + P[0][i]
		self.cvrpLP.addConstr(const5 == 0)

		# constraint 6
		const6 = None
		for i in range(1, self.n):
			if(const6 == None):
				const6 = R[i][0]
			else:
				const6 = const6 + R[i][0]
		self.cvrpLP.addConstr(const6 == 0)

		# constraint 7
		for i in range(self.n):
			for j in range(self.n):
				self.cvrpLP.addConstr(R[i][j] + P[i][j] <= self.capacityOfVehicle * x[i][j])

		# constraint 8
		for i in range(1, self.n):
			self.cvrpLP.addConstr(x[0][i] <= self.numberOfVehicles)

	def solve(self):
		status = self.cvrpLP.optimize()
		print(status)
	
	def getResult(self):
		print("Objective value: ", self.cvrpLP.ObjVal)
		for v in self.cvrpLP.getVars():
			print(v.varName, " = ", v.x)
		return self.cvrpLP

## E-N13-K4.vrp - Christofides and Eilon

In [ ]:
# costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],   
# [9,0,21,22,36,52,24,51,36,37,41,30,20],    
# [14,21,0,25,38,5,31,7,36,43,29,7,6],    
# [23,22,25,0,42,12,35,17,44,31,31,11,6],
# [32,36,38,42,0,22,37,16,46,37,29,13,14],   
# [50,52,5,12,22,0,41,23,10,39,9,17,16],   
# [21,24,31,35,37,41,0,26,21,19,10,25,12],  
# [49,51,7,17,16,23,26,0,30,28,16,27,12],   
# [30,36,36,44,46,10,21,30,0,25,22,10,20],    
# [27,37,43,31,37,39,19,28,25,0,20,16,8],   
# [35,41,29,31,29,9,10,16,22,20,0,10,10],   
# [28,30,7,11,13,17,25,27,10,16,10,0,10],
# [18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

# len(costMatrix)

def read_vrpspd_file(file_path):
    with open(file_path, "r") as f:
        lines = [line.strip() for line in f.readlines() if line.strip() and "~" not in line]

    cost_matrix = []
    delivery = []
    pickup = []
    vehicle_capacities = []

    i = 0

    # 1. Read cost matrix
    if "Cost matrix" in lines[i]:
        i += 1

    # đọc từng dòng số -> ma trận
    while i < len(lines) and "Delivery quantities" not in lines[i]:
        row = list(map(float, lines[i].split()))
        cost_matrix.append(row)
        i += 1

    # 2. Read delivery
    if "Delivery quantities" in lines[i]:
        i += 1

    delivery = list(map(int, map(float, lines[i].split())))
    i += 1

    # 3. Read pickup
    if "Pick-up quantities" in lines[i]:
        i += 1

    pickup = list(map(int, map(float, lines[i].split())))
    i += 1

    # 4. Read vehicle capacity (LIST of capacities)
    if "Vehicle capacity" in lines[i]:
        i += 1
        vehicle_capacities = list(map(int, lines[i].split()))
        i += 1

    # === ADD DEPOT ===
    delivery = [0] + delivery
    pickup = [0] + pickup

    # === FINAL VARIABLES ===
    numberOfVehicles = len(vehicle_capacities)

    # if all vehicles have same capacity -> use that as capacityOfVehicle
    if len(set(vehicle_capacities)) == 1:
        capacityOfVehicle = vehicle_capacities[0]
    else:
        # different capacity vehicles
        capacityOfVehicle = max(vehicle_capacities)

    return cost_matrix, delivery, pickup, vehicle_capacities, capacityOfVehicle, numberOfVehicles


# ===== USAGE =====
file_path = "D:\\VRP\\VRPSPD\\class4\\R101_20_08.txt"   # <- thay đường dẫn file thật của bạn
costMatrix, demand, pickup, vehicle_caps, capacityOfVehicle, numberOfVehicles = read_vrpspd_file(file_path)

print("\n✅ costMatrix =", costMatrix)
print("\n✅ demand =", demand)
print("\n✅ pickup =", pickup)
print("\n✅ vehicle_capacities =", vehicle_caps)
print("✅ capacityOfVehicle =", capacityOfVehicle)
print("✅ numberOfVehicles =", numberOfVehicles)


In [ ]:
# demand = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100] 
# pickup = [0, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
# capacityOfVehicle = 6000
# numberOfVehicles = 4

In [ ]:
lp2 = VRPSDP_GUROBI(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp2.solve()
result = lp2.getResult()

In [ ]:
# Use output to get path
# gets variables as (x,y) coordinates
variables = []
for v in result.getVars():
    if('x' in v.varName and v.x == 1):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
        variables.append((int(temp[0]),int(temp[1])))
print(variables)

variablesR = {}
for v in result.getVars():
    if('R' in v.varName and v.x > 0):
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
#         variablesR[((xCoordinates[int(temp[0])],yCoordinates[int(temp[0])]), (xCoordinates[int(temp[1])],yCoordinates[int(temp[1])]))] = round(v.x, 2)
        variablesR[(xCoordinates[int(temp[1])],yCoordinates[int(temp[1])])] = round(v.x, 2)
print("variablesR", variablesR)

variablesP = {}
for v in result.getVars():
    if('P' in v.varName and v.x > 0):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
#         variablesP[((xCoordinates[int(temp[0])],yCoordinates[int(temp[0])]), (xCoordinates[int(temp[1])],yCoordinates[int(temp[1])]))] = round(v.x, 2)
        variablesP[(xCoordinates[int(temp[1])],yCoordinates[int(temp[1])])] = round(v.x, 2)
print("variablesP",variablesP)

#recursive calls for getting the path
def recursiveList(start, L, X, c):
    if(start in c):
        return X
    for item in L:
        if(item[0] == start):
            X.append(item)
            c.append(start)
            return recursiveList(item[1], L, X, c)
    return X

pathList = []
setList = []
start = 0
for v in variables:
    if(v[0] == start):
        path = recursiveList(v[1], variables, [v], [start])
        print(path)
        pathList.append(path)
        set1 = []
        for i in path:
            set1.append(i[0])
            set1.append(i[1])
        a = list(set(set1))
        if(len(a)>1):
            setList.append(sorted(a))
            
print(setList)

In [ ]:
# Visualization
plt.figure(figsize=(12,12))
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

coordinateList = []
for s in setList:
    coordinate = []
    for j in s:
        coordinate.append([float(xCoordinates[j]), float(yCoordinates[j])])
    coordinateList.append(coordinate)
# print(coordinateList)

def addToPlot(L):
    x_val = [x[0] for x in L]
    y_val = [x[1] for x in L]

    r = random.random()
    b = random.random()
    g = random.random()
    newColor = (r, g, b)
    
    plt.scatter(x_val,y_val, c=newColor, edgecolor='black', linewidth=2, s=1000, marker='h')
    ax = plt.axes()
    
    length = len(L)-1
    
    for i in range(length):
        ax.arrow(L[i][0],  #x1
                    L[i][1],  # y1
                    L[i+1][0]-L[i][0], # x2 - x1
                    L[i+1][1]-L[i][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
        aS = ""
        if((int(L[i+1][0]),int(L[i+1][1]))) in variablesR:
            aS += "D:" + str(variablesR[(int(L[i+1][0]),int(L[i+1][1]))]) + "\n"
        if((int(L[i+1][0]),int(L[i+1][1]))) in variablesP:
            aS += "P:" + str(variablesP[(int(L[i+1][0]),int(L[i+1][1]))]) + "\n"
        
        ax.annotate(aS, xy=(L[i+1][0], L[i+1][1]), xytext=(-20,20), 
            textcoords='offset points', ha='center', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                            color='red'))
    
    ax.arrow(L[-1][0],  #x1
                    L[-1][1],  # y1
                    L[0][0]-L[-1][0], # x2 - x1
                    L[0][1]-L[-1][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
for i in range(len(coordinateList)):
#     print(coordinateList[i])
    addToPlot(coordinateList[i])
plt.scatter(coordinateList[0][0][0],coordinateList[0][0][1], c='black', edgecolor='black', linewidth='2', s=1000, marker='h')
plt.show()

## E-N22-K4.vrp - Christofides and Eilon

In [ ]:
# E-n22-k4.vrp = CE22P

xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500, 700]
pickup = [0, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500]
capacityOfVehicle = 6000
numberOfVehicles = 7

In [ ]:
lp = VRPSDP_GUROBI(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)

In [ ]:
lp.solve()
result = lp.getResult()

In [ ]:
# Use output to get path
# gets variables as (x,y) coordinates
variables = []
for v in result.getVars():
    if('x' in v.varName and v.x == 1):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
        variables.append((int(temp[0]),int(temp[1])))
print(variables)

variablesR = {}
for v in result.getVars():
    if('R' in v.varName and v.x > 0):
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
#         variablesR[((xCoordinates[int(temp[0])],yCoordinates[int(temp[0])]), (xCoordinates[int(temp[1])],yCoordinates[int(temp[1])]))] = round(v.x, 2)
        variablesR[(xCoordinates[int(temp[1])],yCoordinates[int(temp[1])])] = round(v.x, 2)
print("variablesR", variablesR)

variablesP = {}
for v in result.getVars():
    if('P' in v.varName and v.x > 0):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
#         variablesP[((xCoordinates[int(temp[0])],yCoordinates[int(temp[0])]), (xCoordinates[int(temp[1])],yCoordinates[int(temp[1])]))] = round(v.x, 2)
        variablesP[(xCoordinates[int(temp[1])],yCoordinates[int(temp[1])])] = round(v.x, 2)
print("variablesP",variablesP)

#recursive calls for getting the path
def recursiveList(start, L, X, c):
    if(start in c):
        return X
    for item in L:
        if(item[0] == start):
            X.append(item)
            c.append(start)
            return recursiveList(item[1], L, X, c)
    return X

pathList = []
setList = []
start = 0
for v in variables:
    if(v[0] == start):
        path = recursiveList(v[1], variables, [v], [start])
        print(path)
        pathList.append(path)
        set1 = []
        for i in path:
            set1.append(i[0])
            set1.append(i[1])
        a = list(set(set1))
        if(len(a)>1):
            setList.append(sorted(a))
            
print(setList)

In [ ]:
# Visualization
plt.figure(figsize=(12,12))
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

coordinateList = []
for s in setList:
    coordinate = []
    for j in s:
        coordinate.append([float(xCoordinates[j]), float(yCoordinates[j])])
    coordinateList.append(coordinate)
# print(coordinateList)

def addToPlot(L):
    x_val = [x[0] for x in L]
    y_val = [x[1] for x in L]

    r = random.random()
    b = random.random()
    g = random.random()
    newColor = (r, g, b)
    
    plt.scatter(x_val,y_val, c=newColor, edgecolor='black', linewidth=2, s=1000, marker='h')
    ax = plt.axes()
    
    length = len(L)-1
    
    for i in range(length):
        ax.arrow(L[i][0],  #x1
                    L[i][1],  # y1
                    L[i+1][0]-L[i][0], # x2 - x1
                    L[i+1][1]-L[i][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
        aS = ""
        if((int(L[i+1][0]),int(L[i+1][1]))) in variablesR:
            aS += "D:" + str(variablesR[(int(L[i+1][0]),int(L[i+1][1]))]) + "\n"
        if((int(L[i+1][0]),int(L[i+1][1]))) in variablesP:
            aS += "P:" + str(variablesP[(int(L[i+1][0]),int(L[i+1][1]))]) + "\n"
        
        ax.annotate(aS, xy=(L[i+1][0], L[i+1][1]), xytext=(-20,20), 
            textcoords='offset points', ha='center', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                            color='red'))
    
    ax.arrow(L[-1][0],  #x1
                    L[-1][1],  # y1
                    L[0][0]-L[-1][0], # x2 - x1
                    L[0][1]-L[-1][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
for i in range(len(coordinateList)):
#     print(coordinateList[i])
    addToPlot(coordinateList[i])
plt.scatter(coordinateList[0][0][0],coordinateList[0][0][1], c='black', edgecolor='black', linewidth='2', s=1000, marker='h')
plt.show()

## E-N30-K4.vrp - Christofides and Eilon

In [ ]:
xCoordinates = [162, 218, 218, 201, 214, 224, 210, 104, 126, 119, 129, 126, 125, 116, 126, 125, 119, 115, 153, 175, 180, 159, 188, 152, 215, 212, 188, 207, 184, 207]
yCoordinates = [354, 382, 358, 370, 371, 370, 382, 354, 338, 340, 349, 347, 346, 355, 335, 355, 357, 341, 351, 363, 360, 331, 357, 349, 389, 394, 393, 406, 410, 392]

costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))

demand = [0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150, 1000]
pickup = [0, 0, 300, 3100, 125, 100, 200, 150, 150, 450, 300, 100, 950, 125, 150, 150, 550, 150, 100, 150, 400, 300, 1500, 100, 300, 500, 800, 300, 100, 150]
capacityOfVehicle = 4500
numberOfVehicles = 4




In [ ]:
lp = VRPSDP_GUROBI(costMatrix, demand, pickup, numberOfVehicles, capacityOfVehicle)
lp.solve()
result = lp.getResult()

In [ ]:
# Use output to get path
# gets variables as (x,y) coordinates
variables = []
for v in result.getVars():
    if('x' in v.varName and v.x == 1):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
        variables.append((int(temp[0]),int(temp[1])))
print(variables)

variablesR = {}
for v in result.getVars():
    if('R' in v.varName and v.x > 0):
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
#         variablesR[((xCoordinates[int(temp[0])],yCoordinates[int(temp[0])]), (xCoordinates[int(temp[1])],yCoordinates[int(temp[1])]))] = round(v.x, 2)
        variablesR[(xCoordinates[int(temp[1])],yCoordinates[int(temp[1])])] = round(v.x, 2)
print("variablesR", variablesR)

variablesP = {}
for v in result.getVars():
    if('P' in v.varName and v.x > 0):
        # print(v.name, " = ", v.varValue)
        temp = (v.varName.split('(')[1].split(')')[0].split(','))
#         variablesP[((xCoordinates[int(temp[0])],yCoordinates[int(temp[0])]), (xCoordinates[int(temp[1])],yCoordinates[int(temp[1])]))] = round(v.x, 2)
        variablesP[(xCoordinates[int(temp[1])],yCoordinates[int(temp[1])])] = round(v.x, 2)
print("variablesP",variablesP)

#recursive calls for getting the path
def recursiveList(start, L, X, c):
    if(start in c):
        return X
    for item in L:
        if(item[0] == start):
            X.append(item)
            c.append(start)
            return recursiveList(item[1], L, X, c)
    return X

pathList = []
setList = []
start = 0
for v in variables:
    if(v[0] == start):
        path = recursiveList(v[1], variables, [v], [start])
        print(path)
        pathList.append(path)
        set1 = []
        for i in path:
            set1.append(i[0])
            set1.append(i[1])
        a = list(set(set1))
        if(len(a)>1):
            setList.append(sorted(a))
            
print(setList)

In [ ]:
# Visualization
plt.figure(figsize=(12,12))
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)

coordinateList = []
for s in setList:
    coordinate = []
    for j in s:
        coordinate.append([float(xCoordinates[j]), float(yCoordinates[j])])
    coordinateList.append(coordinate)
# print(coordinateList)

def addToPlot(L):
    x_val = [x[0] for x in L]
    y_val = [x[1] for x in L]

    r = random.random()
    b = random.random()
    g = random.random()
    newColor = (r, g, b)
    
    plt.scatter(x_val,y_val, c=newColor, edgecolor='black', linewidth=2, s=1000, marker='h')
    ax = plt.axes()
    
    length = len(L)-1
    
    for i in range(length):
        ax.arrow(L[i][0],  #x1
                    L[i][1],  # y1
                    L[i+1][0]-L[i][0], # x2 - x1
                    L[i+1][1]-L[i][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
        aS = ""
        if((int(L[i+1][0]),int(L[i+1][1]))) in variablesR:
            aS += "D:" + str(variablesR[(int(L[i+1][0]),int(L[i+1][1]))]) + "\n"
        if((int(L[i+1][0]),int(L[i+1][1]))) in variablesP:
            aS += "P:" + str(variablesP[(int(L[i+1][0]),int(L[i+1][1]))]) + "\n"
        
        ax.annotate(aS, xy=(L[i+1][0], L[i+1][1]), xytext=(-20,20), 
            textcoords='offset points', ha='center', va='bottom',
            bbox=dict(boxstyle='round,pad=0.2', fc='yellow', alpha=0.3),
            arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5', 
                            color='red'))
    
    ax.arrow(L[-1][0],  #x1
                    L[-1][1],  # y1
                    L[0][0]-L[-1][0], # x2 - x1
                    L[0][1]-L[-1][1], # y2 - y1
                    width=0.1, head_width=0.6, head_length=0.6, color=newColor, length_includes_head=True,
                    )
for i in range(len(coordinateList)):
#     print(coordinateList[i])
    addToPlot(coordinateList[i])
plt.scatter(coordinateList[0][0][0],coordinateList[0][0][1], c='black', edgecolor='black', linewidth='2', s=1000, marker='h')
plt.show()